In [2]:
import pandas as pd
import numpy as np
from array import array
import datetime as dt

from itertools import chain


from support import *
from features.feature_ts import genX
from experiment.algorithms.cluster_prep import *

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\os.py
c:\Users\Jason\thesis_project


In [40]:
X = genX([1994,2014], drop_0 = True) # Load dataframe

dropping all zero rows


In [26]:
# df_winter.set_index(['ProfileID'], inplace = True)


In [57]:
# Check which households
df_winter

,ProfileID,date,0,1,2,3,4,5,6,7,...,17,18,19,20,21,22,23,month,season,daytype
0,1,1994-06-18,3.691667,0.800000,0.508333,2.183333,0.741667,0.658333,1.991667,0.700000,...,3.916667,9.241667,3.275000,6.325000,2.816667,0.925000,2.025000,6,winter,Saturday
1,1,1994-06-19,0.808333,0.500000,2.075000,0.691667,0.325000,2.075000,0.691667,0.316667,...,1.141667,7.283333,5.391667,6.441667,9.716667,7.500000,0.841667,6,winter,Sunday
2,1,1994-06-20,1.058333,1.625000,0.341667,2.100000,1.225000,1.058333,4.808333,4.091667,...,5.541667,5.275000,6.533333,4.708333,10.433333,9.508333,4.408333,6,winter,weekday
3,1,1994-06-21,0.425000,2.350000,0.550000,0.291667,2.100000,0.591667,4.066667,2.791667,...,1.833333,3.466667,8.875000,10.850000,12.908333,9.241667,2.625000,6,winter,weekday
4,1,1994-06-22,0.741667,2.325000,0.583333,1.191667,1.225000,0.575000,4.066667,3.350000,...,6.291667,6.491667,4.458333,13.125000,10.750000,6.200000,1.350000,6,winter,weekday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252795,12029071,2014-08-27,0.320000,0.286667,0.260000,0.240000,0.220000,1.233333,3.346667,0.240000,...,0.720000,2.006667,1.000000,3.580000,1.466667,0.280000,0.400000,8,winter,weekday
3252796,12029071,2014-08-28,0.266667,0.266667,0.366667,0.286667,0.240000,1.053333,3.920000,1.273333,...,0.246667,1.606667,1.333333,3.860000,1.586667,0.293333,0.273333,8,winter,weekday
3252797,12029071,2014-08-29,0.273333,0.273333,0.346667,0.300000,0.273333,1.173333,3.093333,0.813333,...,0.400000,2.200000,1.513333,3.893333,1.453333,0.373333,0.206667,8,winter,weekday
3252798,12029071,2014-08-30,0.300000,0.313333,0.200000,0.246667,0.353333,0.193333,0.160000,3.140000,...,1.046667,2.853333,1.686667,1.313333,2.100000,0.560000,0.240000,8,winter,Saturday


In [69]:

# Remove all households with less than three winter months
winter_months_ids = []

for i in df_winter['ProfileID'].unique():
    if len(df_winter[df_winter['ProfileID'] == i]['month'].unique()) == 3:

        winter_months_ids.append(i)

import pickle

# save list of houeholds with complete winter months
with open("winter_months_ids", "wb") as fp:   #Pickling
   pickle.dump(winter_months_ids, fp)



In [41]:
# data = pd.read_pickle('sorted_households.pkl') # Read pickle file
X.reset_index(inplace = True)
# # Extract household IDs
# household_ids = list(data.values())
# householdIDs =  [l.tolist() for l in household_ids]
# householdIDs = list(map(int, chain.from_iterable(householdIDs)))

# Select rows based on condition 
# df = X[X['ProfileID'].isin(householdIDs)] 
df = X.copy()

df['month'] = df.date.dt.month
df['season'] = df['month'].apply(lambda x: 'winter' if x in [6, 7, 8] else 'summer') 
df_winter = df[df['season'] == 'winter'] # Create dataframe with all the winter months, excluding weekends


weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']

# extract weekdays
df_winter['day_names'] = df_winter.date.dt.day_name()
df_winter['daytype'] = df_winter.day_names.where(~df_winter.day_names.isin(weekdays), 'weekday')

df_winter.drop(['day_names'], axis = 1, inplace = True)

df_winter_weekdays  = df_winter[df_winter['daytype'] == 'weekday'] # Create dataframe with only weekdays

df_winter_weekdays.drop(['month', 'season','daytype'], axis = 1, inplace = True)
df_winter_weekdays.set_index(['ProfileID','date'], inplace=True)

# df_winter_weekdays.to_csv("Winter_Weekdays_Dataframe.csv")


<ipython-input-41-8aafd821d395>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-41-8aafd821d395>:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Remove all weekends and winter months

In [67]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']

# extract weekdays
df['day_names'] = df.date.dt.day_name()
df['daytype'] = df.day_names.where(~df.day_names.isin(weekdays), 'weekday')

df.drop(['day_names'], axis = 1, inplace = True)

df2 = df[df['daytype'] == 'weekday'] # Create dataframe with only weekdays

# df2['month'] = df2.date.dt.month
df2['season'] = df2['month'].apply(lambda x: 'winter' if x in [6, 7, 8, 9] else 'summer') 
df3 = df2[df2['season'] == 'winter'] # Create dataframe with all the winter months, excluding weekends


<ipython-input-67-1bfdb0562e77>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-67-1bfdb0562e77>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Calculate average monthly consumption for entire year

In [14]:
X.set_index(['ProfileID', 'date'], inplace = True)
Xdd_A = X.sum(axis=1)
Xdd = Xdd_A*230/1000
Xyearly = resampleProfiles(Xdd, interval='A', aggfunc='sum')
XmonthlyPower = resampleProfiles(Xdd, interval='M', aggfunc='sum')
Xamd = resampleProfiles(XmonthlyPower, interval='A', aggfunc='mean').reset_index().groupby('ProfileID').mean()
Xamd.columns=['AMC_yearly']

In [120]:
Xyearly # Total yearly consumption

0
ProfileID date                    
1         1994-12-31   1900.723862
          1995-12-31   1932.008074
2         1996-12-31     42.477603
3         1997-12-31    328.283568
          1998-12-31   8907.553704
...                            ...
12029049  2014-12-31    275.032428
12029058  2014-12-31  10163.465261
12029062  2014-12-31   4726.686040
12029066  2014-12-31  14984.583685
12029071  2014-12-31    213.850939

[22147 rows x 1 columns]

In [5]:
Xamd # Average monthly consumption 

,AMC_yearly
ProfileID,
1,296.766663
2,10.619401
3,569.030589
4,404.916203
5,510.245011
...,...
12029049,68.758107
12029058,2540.866315
12029062,1181.671510


## Calculate average monthly consumption for winter

In [9]:
df_winter.drop(['month','season','daytype'],axis = 1, inplace = True)

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
df_winter

,ProfileID,date,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,1,1994-06-18,3.691667,0.800000,0.508333,2.183333,0.741667,0.658333,1.991667,0.700000,...,1.450000,1.133333,1.666667,3.916667,9.241667,3.275000,6.325000,2.816667,0.925000,2.025000
1,1,1994-06-19,0.808333,0.500000,2.075000,0.691667,0.325000,2.075000,0.691667,0.316667,...,2.125000,2.616667,2.358333,1.141667,7.283333,5.391667,6.441667,9.716667,7.500000,0.841667
2,1,1994-06-20,1.058333,1.625000,0.341667,2.100000,1.225000,1.058333,4.808333,4.091667,...,1.433333,2.641667,2.116667,5.541667,5.275000,6.533333,4.708333,10.433333,9.508333,4.408333
3,1,1994-06-21,0.425000,2.350000,0.550000,0.291667,2.100000,0.591667,4.066667,2.791667,...,3.150000,1.825000,4.016667,1.833333,3.466667,8.875000,10.850000,12.908333,9.241667,2.625000
4,1,1994-06-22,0.741667,2.325000,0.583333,1.191667,1.225000,0.575000,4.066667,3.350000,...,1.500000,0.933333,2.791667,6.291667,6.491667,4.458333,13.125000,10.750000,6.200000,1.350000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252825,12029071,2014-09-26,1.366667,0.273333,0.273333,0.260000,0.346667,2.353333,0.313333,1.313333,...,0.200000,0.306667,0.260000,0.206667,0.386667,0.973333,1.946667,3.966667,0.613333,0.293333
3252826,12029071,2014-09-27,0.326667,0.346667,0.306667,0.286667,0.266667,1.346667,1.753333,0.506667,...,0.326667,0.320000,0.320000,0.320000,0.640000,1.893333,1.273333,2.766667,0.533333,0.340000
3252827,12029071,2014-09-28,0.346667,0.286667,0.286667,0.393333,0.293333,0.246667,0.220000,0.523636,...,0.400000,0.246667,0.326667,0.373333,0.900000,2.173333,3.073333,2.453333,0.513333,0.400000
3252828,12029071,2014-09-29,0.400000,0.400000,0.193333,0.246667,1.733333,2.886667,0.440000,0.340000,...,0.853333,0.773333,0.320000,0.406667,1.566667,0.686667,2.273333,1.620000,0.506667,0.400000


In [15]:
# df_winter.drop(['month','season'], axis = 1, inplace = True)
df_winter.set_index(['ProfileID','date'], inplace=True)
Xdd_winter_A = df_winter.sum(axis=1)
Xdd_winter = Xdd_winter_A*230/1000
Xyearly_winter = resampleProfiles(Xdd_winter, interval='A', aggfunc='sum')
XmonthlyPower_winter = resampleProfiles(Xdd_winter, interval='M', aggfunc='sum')
Xamd_winter = resampleProfiles(XmonthlyPower_winter, interval='A', aggfunc='mean').reset_index().groupby('ProfileID').mean()
Xamd_winter.columns=['AMC_winter']

In [16]:
Xamd_winter # Average monthly consumption for winter months

,AMC_winter
ProfileID,
1,111.114074
2,11.693588
3,994.282506
4,619.423681
5,884.767180
...,...
12029049,76.771501
12029058,2866.501198
12029062,1359.023402


In [17]:
Xamd

,AMC_yearly
ProfileID,
1,296.766663
2,10.619401
3,569.030589
4,404.916203
5,510.245011
...,...
12029049,68.758107
12029058,2540.866315
12029062,1181.671510


# Create new dataframe


In [153]:
df_amd = pd.merge(Xamd_winter,Xamd,right_index = True, left_index = True)

In [154]:
df_amd

,AMC_winter,AMC_yearly
ProfileID,,
1,111.114074,296.766663
3,994.282506,569.030589
4,619.423681,404.916203
5,884.767180,510.245011
6,538.049648,331.499595
...,...,...
12029019,412.007433,448.671829
12029032,1869.001213,1370.898039
12029036,470.454520,406.580486


In [155]:
df_amd.to_csv('AMC_dataframe.csv')

In [ ]:
df_amd.to_csv('AMC_dataframe.csv')

In [74]:
df_tempo = pd.read_csv('AMC_dataframe.csv')